##Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!git clone -b legacy https://github.com/OpenNMT/OpenNMT-py
!pip install OpenNMT-py
import os
outdir = 'drive/MyDrive/GermanMccurdyComparison/output'
if not os.path.exists(outdir):
    os.makedirs(outdir)

!pip install scipy
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17355, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 17355 (delta 189), reused 156 (delta 90), pack-reused 17044
Receiving objects: 100% (17355/17355), 273.55 MiB | 22.83 MiB/s, done.
Resolving deltas: 100% (12489/12489), done.
     |████████████████████████████████| 215kB 14.2MB/s 
     |████████████████████████████████| 14.3MB 8.6MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 645kB 48.2MB/s 
     |████████████████████████████████| 748.8MB 23kB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 1.2MB 54.9MB/s 
  Created whee

##Generate Data

In [ ]:
datasizes = [f'split_{i}' for i in range(25)]

In [ ]:
import numpy as np
import random
import pandas as pd
import operator

random.seed(1)

Data = np.load('drive/MyDrive/GermanMccurdyComparison/german_data.npy', allow_pickle=True).item() # Format lemma: {S:(wordform, gender, phon, idNum); P:(...)}
genders = {'1':'MAS', '2':'FEM','3':'NTR'} 			# 5. GendNum (1 = M, 2 = F, 3 = N)
# print(Data)
# raise RuntimeError
# print(len(Data)) 20k words
# Get the most frequent n lemmas
n = 5000
sortedLemmas = sorted(Data, key = lambda x: int(Data[x]['f']), reverse = True)[:n]

nouns = []
n2f = {}
for lemma in sortedLemmas:
 data = Data[lemma] 
 gender = genders[data['P'][1]]
 plural = data['P'][0].lower()
 singular = data['S'][0].lower()
 n2f[singular] = int(data['f'])
 nouns.append((singular,gender,plural))

for i, path in enumerate([f'drive/MyDrive/GermanMccurdyComparison/processed_data/{split}' for split in datasizes]):
  if not os.path.exists(path):
    os.makedirs(path)
  random.seed(i)
  random.shuffle(nouns)
  with open(f'{path}/german-src-train.txt','w') as src:
    with open(f'{path}/german-tgt-train.txt','w') as tgt:
      for s,g,p in nouns[0:4000]:
        src.write(g+' <s> '+' '.join(list(s))+'\n')
        tgt.write(' '.join(list(p))+'\n')

  with open(f'{path}/german-src-val.txt','w') as src:
    with open(f'{path}/german-tgt-val.txt','w') as tgt:
      for s,g,p in nouns[4000:4500]:
        src.write(g+' <s> '+' '.join(list(s))+'\n')
        tgt.write(' '.join(list(p))+'\n')
  
  with open(f'{path}/german-src-test.txt','w') as src:
    with open(f'{path}/german-tgt-test.txt','w') as tgt:
      for s,g,p in nouns[4500:5000]:
        src.write(g+' <s> '+' '.join(list(s))+'\n')
        tgt.write(' '.join(list(p))+'\n')

csv = pd.read_csv(f'drive/MyDrive/GermanMccurdyComparison/study1_data.csv')
csv = csv[csv['Q_Task']=='production']

wugs=csv['Q_Lemma'].unique()
print(wugs)

with open(f'drive/MyDrive/GermanMccurdyComparison/mccurdy-test.txt','w') as src:
  for s in wugs:
    src.write('NTR'+' <s> '+' '.join(list(s))+'\n')


['bral' 'kach' 'klot' 'mur' 'nuhl' 'pind' 'pisch' 'pund' 'raun' 'spand'
 'vag' 'spert' 'bnaupf' 'bneik' 'bnöhk' 'fnahf' 'fneik' 'fnöhk' 'pläk'
 'plaupf' 'pleik' 'pnähf' 'pröng' 'snauk']


##Preprocess data

In [ ]:
for datasize in datasizes:
  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{datasize}'
  !python OpenNMT-py/preprocess.py -train_src $datadir/german-src-train.txt -train_tgt $datadir/german-tgt-train.txt -valid_src $datadir/german-src-val.txt -valid_tgt $datadir/german-tgt-val.txt -save_data $datadir/processed

[2021-02-03 04:26:36,463 INFO] Extracting features...
[2021-02-03 04:26:36,465 INFO]  * number of source features: 0.
[2021-02-03 04:26:36,465 INFO]  * number of target features: 0.
[2021-02-03 04:26:36,465 INFO] Building `Fields` object...
[2021-02-03 04:26:36,465 INFO] Building & saving training data...
[2021-02-03 04:26:36,481 INFO] Building shard 0.
[2021-02-03 04:26:36,585 INFO]  * saving 0th train data shard to drive/MyDrive/GermanMccurdyComparison/processed_data/split_0/processed.train.0.pt.
[2021-02-03 04:26:36,777 INFO]  * tgt vocab size: 30.
[2021-02-03 04:26:36,777 INFO]  * src vocab size: 32.
[2021-02-03 04:26:36,783 INFO] Building & saving validation data...
[2021-02-03 04:26:36,797 INFO] Building shard 0.
[2021-02-03 04:26:36,807 INFO]  * saving 0th valid data shard to drive/MyDrive/GermanMccurdyComparison/processed_data/split_0/processed.valid.0.pt.
[2021-02-03 04:26:37,832 INFO] Extracting features...
[2021-02-03 04:26:37,834 INFO]  * number of source features: 0.
[2021

##Train RNN

In [ ]:
for datasize in datasizes:
  epochs, n_examples, batchsize,  = 50, 4000, 20
  steps = str(int(epochs * n_examples / batchsize))

  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{datasize}'
  rnn_modelpath = f'{outdir}/german_rnn_model_{datasize}'
  rnn_train_args = ' '.join([
    f'-data {datadir}/processed',
    '-save_model '+rnn_modelpath,
    '-enc_layers 2',
    '-dec_layers 2',
    '-rnn_size 100',
    '-batch_size 20',
    '-word_vec_size 300',
    '-gpu_ranks 0',
    '-train_steps '+steps,
    '-save_checkpoint_steps '+steps
    ])

  !python OpenNMT-py/train.py $rnn_train_args

Streaming output truncated to the last 5000 lines.
[2021-05-01 23:24:59,442 INFO] Step 7300/10000; acc:  94.75; ppl:  1.34; xent: 0.29; lr: 1.00000; 8304/8638 tok/s;    183 sec
[2021-05-01 23:25:00,692 INFO] Step 7350/10000; acc:  94.12; ppl:  1.39; xent: 0.33; lr: 1.00000; 8286/8655 tok/s;    184 sec
[2021-05-01 23:25:01,962 INFO] Step 7400/10000; acc:  95.25; ppl:  1.28; xent: 0.25; lr: 1.00000; 8154/8664 tok/s;    185 sec
[2021-05-01 23:25:01,963 INFO] Loading dataset from drive/MyDrive/GermanMccurdyComparison/processed_data/split_10/processed.train.0.pt
[2021-05-01 23:25:02,005 INFO] number of examples: 4000
[2021-05-01 23:25:03,258 INFO] Step 7450/10000; acc:  96.06; ppl:  1.22; xent: 0.20; lr: 1.00000; 8111/8560 tok/s;    187 sec
[2021-05-01 23:25:04,474 INFO] Step 7500/10000; acc:  94.79; ppl:  1.33; xent: 0.29; lr: 1.00000; 8817/9171 tok/s;    188 sec
[2021-05-01 23:25:05,674 INFO] Step 7550/10000; acc:  93.76; ppl:  1.44; xent: 0.36; lr: 1.00000; 8630/9015 tok/s;    189 sec
[2

## Train Transformer

In [ ]:
for datasize in datasizes:
# for datasize in datasizes[21:]:
  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{datasize}'
  
  transformer_modelpath = f'{outdir}/german_transformer_model_{datasize}'
  steps = 10000
  transformer_train_args = ' '.join(f'''-data {datadir}/processed -gpu_ranks 0
  -save_model {transformer_modelpath} -layers 2 -rnn_size 312
  -word_vec_size 312 -transformer_ff 512 -heads 8 
  -encoder_type transformer -decoder_type transformer -position_encoding 
  -train_steps {steps} -max_generator_batches 2 -dropout 0.1 
  -batch_size 20 -batch_type tokens -normalization tokens 
  -accum_count 2 -optim adam -adam_beta2 0.998 
  -decay_method noam -warmup_steps 1000 -learning_rate 0.05 -max_grad_norm 0 
  -param_init 0 -param_init_glorot -label_smoothing 0.1 
  -valid_steps 1000 -save_checkpoint_steps 5000 -world_size 1'''.split())

  !python OpenNMT-py/train.py $transformer_train_args

[2021-05-01 16:40:14,457 INFO]  * src vocab size = 32
[2021-05-01 16:40:14,458 INFO]  * tgt vocab size = 30
[2021-05-01 16:40:14,458 INFO] Building model...
[2021-05-01 16:40:17,566 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(32, 312, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=312, out_features=312, bias=True)
          (linear_values): Linear(in_features=312, out_features=312, bias=True)
          (linear_query): Linear(in_features=312, out_features=312, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=312, out_features=312,

##Predict

### RNN

In [ ]:
for datasize in datasizes:
  epochs, n_examples, batchsize,  = 50, 4000, 20
  steps = str(int(epochs * n_examples / batchsize))
  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{datasize}'
  rnn_modelpath = f'{outdir}/german_rnn_model_{datasize}'
  rnn_trans_args = ' '.join([
    '-model '+rnn_modelpath+'_step_'+steps+'.pt',
    f'-src {datadir}/german-src-test.txt',
    f'-output {outdir}/german-rnn-{datasize}-pred.txt',
    '-replace_unk',
    '-beam_size 12'
    ])
  !python OpenNMT-py/translate.py $rnn_trans_args
  
  rnn_trans_args = ' '.join([
    '-model '+rnn_modelpath+'_step_'+steps+'.pt',
    f'-src drive/MyDrive/GermanMccurdyComparison/mccurdy-test.txt',
    f'-output {outdir}/german-rnn-{datasize}-mccurdy-pred.txt',
    '-replace_unk',
    '-beam_size 12'
    ])
  !python OpenNMT-py/translate.py $rnn_trans_args

[2021-05-02 00:26:24,370 INFO] Translating shard 0.
[2021-05-02 00:26:27,876 INFO] PRED AVG SCORE: -0.0231, PRED PPL: 1.0234
[2021-05-02 00:26:28,691 INFO] Translating shard 0.
[2021-05-02 00:26:28,752 INFO] PRED AVG SCORE: -0.1059, PRED PPL: 1.1118
[2021-05-02 00:26:29,518 INFO] Translating shard 0.
[2021-05-02 00:26:31,452 INFO] PRED AVG SCORE: -0.0100, PRED PPL: 1.0101
[2021-05-02 00:26:32,243 INFO] Translating shard 0.
[2021-05-02 00:26:32,309 INFO] PRED AVG SCORE: -0.0704, PRED PPL: 1.0729
[2021-05-02 00:26:33,054 INFO] Translating shard 0.
[2021-05-02 00:26:35,491 INFO] PRED AVG SCORE: -0.0139, PRED PPL: 1.0140
[2021-05-02 00:26:36,282 INFO] Translating shard 0.
[2021-05-02 00:26:36,349 INFO] PRED AVG SCORE: -0.0699, PRED PPL: 1.0724
[2021-05-02 00:26:37,113 INFO] Translating shard 0.
[2021-05-02 00:26:40,326 INFO] PRED AVG SCORE: -0.0189, PRED PPL: 1.0191
[2021-05-02 00:26:41,126 INFO] Translating shard 0.
[2021-05-02 00:26:41,189 INFO] PRED AVG SCORE: -0.0691, PRED PPL: 1.0716


### Transformer

In [ ]:
for datasize in datasizes:
  steps = 10000
  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{datasize}'
  transformer_modelpath = f'{outdir}/german_transformer_model_{datasize}'
  transformer_trans_args = ' '.join([
	 '-model '+transformer_modelpath+f'_step_{steps}.pt',
	 f'-src {datadir}/german-src-test.txt',
	 f'-output {outdir}/german-transformer-{datasize}-pred.txt',
	 '-replace_unk',
	 '-beam_size 12'
	])
  !python OpenNMT-py/translate.py $transformer_trans_args
  
  transformer_trans_args = ' '.join([
	 '-model '+transformer_modelpath+f'_step_{steps}.pt',
	 f'-src drive/MyDrive/GermanMccurdyComparison/mccurdy-test.txt',
	 f'-output {outdir}/german-transformer-{datasize}-mccurdy-pred.txt',
	 '-replace_unk',
	 '-beam_size 12'
	])
  !python OpenNMT-py/translate.py $transformer_trans_args


Streaming output truncated to the last 5000 lines.
PRED 50: f a s e r n
PRED SCORE: -1.1074

[2021-05-01 22:31:56,203 INFO] 
SENT 51: ['NTR', '<s>', 'a', 't', 'r', 'i', 'u', 'm']
PRED 51: a t r i e n
PRED SCORE: -2.1489

[2021-05-01 22:31:56,203 INFO] 
SENT 52: ['FEM', '<s>', 'k', 'n', 'e', 'c', 'h', 't', 's', 'c', 'h', 'a', 'f', 't']
PRED 52: k n e c h t s c h a f t e n
PRED SCORE: -0.6091

[2021-05-01 22:31:56,203 INFO] 
SENT 53: ['FEM', '<s>', 'e', 'i', 'n', 'w', 'i', 'r', 'k', 'u', 'n', 'g']
PRED 53: e i n w i r k u n g e n
PRED SCORE: -0.6292

[2021-05-01 22:31:56,203 INFO] 
SENT 54: ['FEM', '<s>', 'k', 'o', 'o', 'p', 'e', 'r', 'a', 't', 'i', 'o', 'n']
PRED 54: k o o p e r a t i o n e n
PRED SCORE: -1.2895

[2021-05-01 22:31:56,203 INFO] 
SENT 55: ['MAS', '<s>', 'b', 'i', 'n', 'd', 'f', 'a', 'd', 'e', 'n']
PRED 55: b i n d f a e d e n
PRED SCORE: -1.8513

[2021-05-01 22:31:56,203 INFO] 
SENT 56: ['MAS', '<s>', 'z', 'i', 'n', 's', 's', 'a', 't', 'z']
PRED 56: z i n s s a e t z e
PR

##Evaluate Results

### Precision, Recall, F1

In [ ]:
from collections import Counter
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

Data = np.load('drive/MyDrive/GermanMccurdyComparison/german_data.npy', allow_pickle=True).item() # Format lemma: {S:(wordform, gender, phon, idNum); P:(...)}
source_to_plural = {}
source_to_inflection = {}
for lemma, data in Data.items():
  plural = data['P'][0].lower()
  singular = data['S'][0].lower()
  source_to_plural[singular] = plural
  if plural[-2:] == singular[-2:]: 
    source_to_inflection[singular] = 'zero'
  elif plural[-1] in ['e','r','s','n']:
    source_to_inflection[singular] = plural[-1]
  else:
    source_to_inflection[singular] = 'other'


def analyze_per_inflection(sources, preds, tgts):
  # Generate list of predicted inflections
  pi = [] # predicted infelctions list
  ti = [] # target infelctions list
  for i, source in enumerate(sources):
    original_singular = source.replace(' ','').split('<s>')[1]
    correct_inflection = source_to_inflection[original_singular]
    ti.append(correct_inflection)
    
    if correct_inflection == 'zero' and preds[i][-1] == source[-1]:
      pi.append('zero')
    elif preds[i][-1] in ['s','e','r','n']:
      pi.append(preds[i][-1])
      
    else:
      pi.append('other')
    
  # pi = [pred[-1] for pred in preds]
  # ti = [tgt[-1] for tgt in tgts]
  # print(precision_recall_fscore_support(ti,pi, labels=['s','e','r','n','zero','other']))
  p,r,f = tuple(list(precision_recall_fscore_support(ti,pi, labels=['s','e','r','n','zero','other'])[:3]))
  l = f"{[f'{p[i]}, {r[i]}, {f[i]}' for i in range(6)]}".replace("'","").replace("[",'').replace("]",'')
  print(l)
  return l
  # return p(s1,s2), p(e1,e2), p(r1,r2), p(n1,n2), p(o1,o2), ''



def frequency_test(train_pairs, srclines, predlines, c=1):
  # Frequency test
  # For each of the verbs in the test set, get the original ending. Then, get the predictions for number of times that ending appears 
  nouns_with_inflections_matching_train = [] 
  for i,noun in enumerate(srclines):
    noun = noun.replace(' ','')
    ending = noun[-c:]
    predicted_inflection = predlines[i].replace(' ','')[-c:] if len(predlines[i]) > 1 else '-'
    train_pairs_with_same_ending = [(s,t) for s,t in train_pairs if s[-c:] == ending]
    # get most frequent inflection for the train data verbs that share the ending
    inflections = [t[-c:] for s,t in train_pairs_with_same_ending]
    if not inflections: continue
    popular_inflection = Counter(inflections).most_common(1)[0][0]
    if predicted_inflection == popular_inflection:
      nouns_with_inflections_matching_train.append((noun, predicted_inflection, popular_inflection))

  print(f'% of inflections (len {c}) that match most popular in training:', len(nouns_with_inflections_matching_train)/len(srclines))
  return len(nouns_with_inflections_matching_train)/len(srclines)

results = []
for i, data_type in enumerate(datasizes):
  print(f'\nData type: {data_type}')
  datadir = f'drive/MyDrive/GermanMccurdyComparison/processed_data/{data_type}'
  
  train_tgt_lines = open(f'{datadir}/german-tgt-train.txt','r').read().splitlines()
  train_src_lines = open(f'{datadir}/german-src-train.txt','r').read().splitlines()
  train_pairs = list(zip([t.replace(' ','') for t in train_src_lines], 
                       [t.replace(' ','') for t in train_tgt_lines]))
  
  for model in ['rnn', 'transformer']:
    srclines = open(f'{datadir}/german-src-test.txt','r').read().splitlines()
    for condition, tgtlines, predlines in [('regular_test', open(f'{datadir}/german-tgt-test.txt','r').read().splitlines(), open(f'{outdir}/german-{model}-{data_type}-pred.txt','r').read().splitlines())]:
      print(f'\n\n{model} {condition} results')
      predlines = [p if len(p) > 1 else '-' for p in predlines]
      tups = list(zip(srclines,tgtlines,predlines))

      m = []
      f = []
      n = []
      for src,tgt,pred in tups:
        # get the learned inflection
        src,tgt,pred = src.strip(), tgt.strip(), pred.strip()
        gender = src.split()[0]
        if gender == 'MAS': m.append((src,tgt,pred))
        elif gender == 'FEM': f.append((src,tgt,pred))
        elif gender == 'NTR': n.append((src,tgt,pred))


      accuracy = sum([t[1][-1]==t[2][-1] for t in tups])/len(tups)
      freq = frequency_test(train_pairs, srclines, predlines)
      print('Test accuracy:', accuracy) 
      ans = analyze_per_inflection(srclines, predlines, tgtlines)
      results.append((data_type, model.upper(), accuracy, freq, ans)) 
      
      
print()
# print(f'data_type, {model.upper()}, condition, test_subset, acc, freq, s, e, r, n, others, others_most_common')
for r in results:
  print(','.join(str(x).replace(',','') for x in r))



Data type: split_0


rnn regular_test results
% of inflections (len 1) that match most popular in training: 0.656
Test accuracy: 0.77
0.1568627450980392, 0.34782608695652173, 0.21621621621621623, 0.8095238095238095, 0.7041420118343196, 0.7531645569620253, 0.3125, 0.25, 0.2777777777777778, 0.9032258064516129, 0.8936170212765957, 0.8983957219251337, 1.0, 0.5, 0.6666666666666666, 0.0, 0.0, 0.0


transformer regular_test results
% of inflections (len 1) that match most popular in training: 0.702
Test accuracy: 0.866
0.8333333333333334, 0.21739130434782608, 0.3448275862068966, 0.7772277227722773, 0.9289940828402367, 0.8463611859838276, 0.6153846153846154, 0.4, 0.4848484848484849, 0.9458483754512635, 0.9290780141843972, 0.9373881932021467, 1.0, 0.5, 0.6666666666666666, 0.0, 0.0, 0.0

Data type: split_1


rnn regular_test results


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


% of inflections (len 1) that match most popular in training: 0.722
Test accuracy: 0.856
0.5, 0.5714285714285714, 0.5333333333333333, 0.766497461928934, 0.937888198757764, 0.8435754189944134, 0.5555555555555556, 0.3333333333333333, 0.4166666666666667, 0.9562043795620438, 0.8618421052631579, 0.9065743944636678, 1.0, 0.5, 0.6666666666666666, 0.0, 0.0, 0.0


transformer regular_test results
% of inflections (len 1) that match most popular in training: 0.732
Test accuracy: 0.89
0.625, 0.35714285714285715, 0.45454545454545453, 0.7894736842105263, 0.9316770186335404, 0.8547008547008548, 0.6666666666666666, 0.4, 0.5, 0.9690721649484536, 0.9276315789473685, 0.9478991596638656, 1.0, 0.5, 0.6666666666666666, 0.0, 0.0, 0.0

Data type: split_2


rnn regular_test results
% of inflections (len 1) that match most popular in training: 0.756
Test accuracy: 0.73
0.42105263157894735, 0.27586206896551724, 0.3333333333333333, 0.8450704225352113, 0.39215686274509803, 0.5357142857142857, 0.5789473684210527, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


% of inflections (len 1) that match most popular in training: 0.754
Test accuracy: 0.844
0.625, 0.1724137931034483, 0.2702702702702703, 0.7164948453608248, 0.9084967320261438, 0.8011527377521614, 0.625, 0.3125, 0.4166666666666667, 0.9409722222222222, 0.9155405405405406, 0.928082191780822, 1.0, 0.3333333333333333, 0.5, 0.0, 0.0, 0.0

Data type: split_3


rnn regular_test results
% of inflections (len 1) that match most popular in training: 0.702
Test accuracy: 0.812
0.42105263157894735, 0.3076923076923077, 0.35555555555555557, 0.722972972972973, 0.7379310344827587, 0.7303754266211604, 0.47619047619047616, 0.5, 0.4878048780487805, 0.9052287581699346, 0.9172185430463576, 0.9111842105263158, 1.0, 0.6, 0.7499999999999999, 0.0, 0.0, 0.0


transformer regular_test results
% of inflections (len 1) that match most popular in training: 0.716
Test accuracy: 0.838
1.0, 0.2692307692307692, 0.42424242424242425, 0.6633165829145728, 0.9103448275862069, 0.7674418604651162, 0.8571428571428571, 0.3, 0.44

### Correlations

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.stats import spearmanr   

csv = pd.read_csv(f'drive/MyDrive/GermanMccurdyComparison/study1_data.csv')
csv = csv[csv['Q_Task']=='production']
wugs= csv['Q_Lemma'].unique() # List of test words
participants = csv['S_Participant_ID'].unique()

# Create dictionary of participant data
# {participant_id: {word: one of ['e','n','s','r','other','zero']}}
csv = csv[['S_Participant_ID','Q_Lemma','Q_PL_Suffix','Q_Lemma_HasRhymes']]
participant_infs = {p:{} for p in participants}
for index, row in csv.iterrows():
  inflection = row['Q_PL_Suffix'][-1] if row['Q_PL_Suffix'] not in ['other','zero'] else row['Q_PL_Suffix']
  participant_infs[row['S_Participant_ID']][row['Q_Lemma']] = inflection

print(participant_infs)

# Source vocab
with open(f'drive/MyDrive/GermanMccurdyComparison/mccurdy-test.txt','r') as source:
  source_lines = [l.replace('NTR <s> ','') for l in source.read().splitlines()]

markers = ['e','n','s','r']
rnn_infs = {}
for datasize in datasizes: # For each data split
  rnn_infs[datasize]={} # Create a dictionary for the RNN participant
  with open(f'{outdir}/german-rnn-{datasize}-mccurdy-pred.txt','r') as preds: # RNN preds for the test
    for i,line in enumerate(preds.read().splitlines()): 
      # Check if it is same as source (i.e zero)
      if line.replace(' ','') == source_lines[i].replace(' ',''):
        rnn_infs[datasize][wugs[i]] = 'zero'
      else:
        rnn_infs[datasize][wugs[i]] = line[-1] if line[-1] in markers else 'other'

transformer_infs = {}
for datasize in datasizes:
  transformer_infs[datasize]={}
  with open(f'{outdir}/german-transformer-{datasize}-mccurdy-pred.txt','r') as preds:
    for i,line in enumerate(preds.read().splitlines()):
      # Check if it is same as source (i.e zero)
      if line.replace(' ','') == source_lines[i].replace(' ',''):
        transformer_infs[datasize][wugs[i]] = 'zero'
      else:
        transformer_infs[datasize][wugs[i]] = line[-1] if line[-1] in markers else 'other'


for inflection in ['e','n','s','r','other','zero']:
  print('Inflection:', inflection)
  participant_vectors = []
  # For each participant, get sub-dictionary of words to inflected forms v={word1 : /en/, ...}
  
  for k, v in participant_infs.items():
    participant_vectors.append([1 if v[wug]==inflection else 0 for wug in wugs])
  particpant_average_vec = np.mean(np.array(participant_vectors),axis=0)

  rnn_vectors = []
  for k, v in rnn_infs.items():
    rnn_vectors.append([1 if v[wug]==inflection else 0 for wug in wugs])
  rnn_average_vec = np.mean(np.array(rnn_vectors),axis=0)

  transformer_vectors = []
  for k, v in transformer_infs.items():
    transformer_vectors.append([1 if v[wug]==inflection else 0 for wug in wugs])
  transformer_average_vec = np.mean(np.array(transformer_vectors),axis=0)
  # print(transformer_vectors)
  # print(transformer_average_vec)

  # Then check how each "type" of particiapnt (RNN/Human) compare to each other

  # a = spearmanr(rnn_average_vec,transformer_average_vec)
  print('Rnn vs Transformer:', spearmanr(rnn_average_vec,transformer_average_vec))
  # b = spearmanr(rnn_average_vec, particpant_average_vec)
  print('Rnn vs Human:', spearmanr(rnn_average_vec, particpant_average_vec))
  # c = spearmanr(particpant_average_vec,transformer_average_vec)
  print('Transformer vs Human:', spearmanr(particpant_average_vec,transformer_average_vec))
  # print(inflection,a,b,c)




{'R_3EaQzuaVgKDZGaJ': {'bral': 'e', 'kach': 'n', 'klot': 'zero', 'mur': 'n', 'nuhl': 'e', 'pind': 'e', 'pisch': 'e', 'pund': 'e', 'raun': 'n', 'spand': 'e', 'vag': 'n', 'spert': 'r', 'bnaupf': 'e', 'bneik': 'e', 'bnöhk': 'e', 'fnahf': 'e', 'fneik': 'e', 'fnöhk': 'e', 'pläk': 'e', 'plaupf': 'e', 'pleik': 'e', 'pnähf': 'e', 'pröng': 'n', 'snauk': 'e'}, 'R_QahTJa0hkfpyXq9': {'bral': 'n', 'kach': 'n', 'klot': 'e', 'mur': 'r', 'nuhl': 'n', 'pind': 'r', 'pisch': 'n', 'pund': 'e', 'raun': 'n', 'spand': 'e', 'vag': 'n', 'spert': 'r', 'bnaupf': 'n', 'bneik': 'n', 'bnöhk': 'e', 'fnahf': 'n', 'fneik': 'n', 'fnöhk': 'n', 'pläk': 'n', 'plaupf': 'e', 'pleik': 'r', 'pnähf': 'r', 'pröng': 'e', 'snauk': 'n'}, 'R_1NabpwGGj73cnSv': {'bral': 's', 'kach': 'r', 'klot': 's', 'mur': 's', 'nuhl': 'e', 'pind': 'r', 'pisch': 'e', 'pund': 'e', 'raun': 'e', 'spand': 'r', 'vag': 'r', 'spert': 'r', 'bnaupf': 'e', 'bneik': 'n', 'bnöhk': 'e', 'fnahf': 'n', 'fneik': 'n', 'fnöhk': 'n', 'pläk': 'e', 'plaupf': 'other', 'p

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
